# importing the libraries

In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.preprocessing import sequence,text
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
train_df=pd.read_csv('../input/nlp-getting-started/train.csv')
test_df=pd.read_csv('../input/nlp-getting-started/test.csv')
sub_df=pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [2]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
train_df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [4]:
train_df.shape

(7613, 5)

In [5]:
train_df=train_df.drop(columns=['keyword','location'],axis=1)
train_df.tail()

,id,text,target
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1
7612,10873,The Latest: More Homes Razed by Northern Calif...,1


In [6]:
train_df.drop(columns='id',axis=1,inplace=True)

In [7]:
train_df.tail()

,text,target
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1
7612,The Latest: More Homes Razed by Northern Calif...,1


In [9]:
train_df['target'].value_counts(normalize=True)

0    0.57034
1    0.42966
Name: target, dtype: float64

In [15]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [16]:
test_df.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [17]:
test_df.shape

(3263, 4)

In [18]:
sub_df.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [19]:
sub_df['target'].value_counts()

0    3263
Name: target, dtype: int64

In [20]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [21]:
voc_size=10000
ps=PorterStemmer()

In [22]:
x=train_df['text']
y=train_df['target']

In [23]:
x

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [24]:
y

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [25]:
len(train_df)

7613

In [29]:
corpus=[]

for i in range(0,len(train_df)):
    print(i)
    review=re.sub('[^a-zA-Z]',' ',x[i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [30]:
corpus

['deed reason earthquak may allah forgiv us',
 'forest fire near la rong sask canada',
 'resid ask shelter place notifi offic evacu shelter place order expect',
 'peopl receiv wildfir evacu order california',
 'got sent photo rubi alaska smoke wildfir pour school',
 'rockyfir updat california hwi close direct due lake counti fire cafir wildfir',
 'flood disast heavi rain caus flash flood street manit colorado spring area',
 'top hill see fire wood',
 'emerg evacu happen build across street',
 'afraid tornado come area',
 'three peopl die heat wave far',
 'haha south tampa get flood hah wait second live south tampa gonna gonna fvck flood',
 'rain flood florida tampabay tampa day lost count',
 'flood bago myanmar arriv bago',
 'damag school bu multi car crash break',
 'man',
 'love fruit',
 'summer love',
 'car fast',
 'goooooooaaaaaal',
 'ridicul',
 'london cool',
 'love ski',
 'wonder day',
 'looooool',
 'way eat shit',
 'nyc last week',
 'love girlfriend',
 'cooool',
 'like pasta',
 '

In [31]:
one_hot_reps1=[text.one_hot(word,voc_size) for word in corpus]
one_hot_reps1

[[3277, 8364, 486, 7128, 1007, 2529, 95],
 [5621, 5687, 9955, 9077, 3492, 9123, 1894],
 [9819, 3361, 9802, 8312, 4676, 7662, 9565, 9802, 8312, 7858, 7952],
 [6506, 2064, 5125, 9565, 7858, 4059],
 [8199, 5980, 5185, 4537, 7553, 5651, 5125, 4758, 7031],
 [8207, 6595, 4059, 2466, 7841, 7392, 4604, 2488, 3796, 5687, 7734, 5125],
 [4525, 6019, 2898, 9922, 1097, 1719, 4525, 9193, 6366, 9343, 8354, 2153],
 [7286, 4454, 908, 5687, 1944],
 [4573, 9565, 3952, 8247, 9698, 9193],
 [3141, 69, 249, 2153],
 [7782, 6506, 4408, 9721, 758, 230],
 [6280,
  4051,
  6895,
  4010,
  4525,
  811,
  961,
  3468,
  5042,
  4051,
  6895,
  7393,
  7393,
  8880,
  4525],
 [9922, 4525, 7212, 3894, 6895, 2025, 2569, 5336],
 [4525, 457, 5174, 1647, 457],
 [5476, 7031, 3492, 7367, 27, 9088, 8129],
 [385],
 [5865, 1843],
 [6309, 5865],
 [27, 4496],
 [4182],
 [2094],
 [2019, 5373],
 [5865, 2257],
 [1069, 2025],
 [9093],
 [4996, 7205, 8640],
 [2726, 4877, 8254],
 [5865, 3540],
 [8311],
 [4785, 6658],
 [1223],
 [4132, 2

In [32]:
sent_length=500

embedded_docs1=sequence.pad_sequences(one_hot_reps1,padding='pre',maxlen=sent_length)
embedded_docs1

array([[   0,    0,    0, ..., 1007, 2529,   95],
       [   0,    0,    0, ..., 3492, 9123, 1894],
       [   0,    0,    0, ..., 8312, 7858, 7952],
       ...,
       [   0,    0,    0, ..., 4982,  327, 9457],
       [   0,    0,    0, ...,  730, 5403, 8862],
       [   0,    0,    0, ..., 4982, 2887, 4380]], dtype=int32)

In [33]:
x=embedded_docs1.copy()
x

array([[   0,    0,    0, ..., 1007, 2529,   95],
       [   0,    0,    0, ..., 3492, 9123, 1894],
       [   0,    0,    0, ..., 8312, 7858, 7952],
       ...,
       [   0,    0,    0, ..., 4982,  327, 9457],
       [   0,    0,    0, ...,  730, 5403, 8862],
       [   0,    0,    0, ..., 4982, 2887, 4380]], dtype=int32)

In [40]:
x.shape

(7613, 500)

In [36]:
y

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [37]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

In [38]:
x_train

array([[   0,    0,    0, ..., 2950, 2472, 2544],
       [   0,    0,    0, ..., 4010, 9994, 6347],
       [   0,    0,    0, ..., 5715, 9101, 1592],
       ...,
       [   0,    0,    0, ..., 8931, 9018, 4525],
       [   0,    0,    0, ..., 4982, 1319, 9037],
       [   0,    0,    0, ..., 7243, 1725,  132]], dtype=int32)

In [39]:
x_test

array([[   0,    0,    0, ...,  450, 2352,  925],
       [   0,    0,    0, ..., 4785,  908, 5042],
       [   0,    0,    0, ..., 4982, 1725, 9152],
       ...,
       [   0,    0,    0, ..., 9881, 4982, 1411],
       [   0,    0,    0, ..., 9881, 4982, 3233],
       [   0,    0,    0, ..., 6108, 4051,  283]], dtype=int32)

In [41]:
y_train

4834    1
1381    1
5924    0
6961    0
1505    1
       ..
1513    0
6259    0
4640    0
4133    1
2321    1
Name: target, Length: 6090, dtype: int64

In [42]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(6090, 500) (1523, 500) (6090,) (1523,)


In [43]:
feature_vector=500

In [46]:
import warnings
warnings.filterwarnings('ignore')

In [47]:
nn1=models.Sequential([
    
    layers.Embedding(voc_size,feature_vector,input_length=sent_length),
    layers.Conv1D(64,8,activation='relu'),
    layers.MaxPooling1D(2),
    layers.Dropout(0.8),
    
    layers.Dense(40,activation='relu'),
    layers.Dropout(0.9),
    
    layers.Dense(29,activation='relu'),
    layers.GlobalMaxPooling1D(),
    
    layers.Dense(1,activation='sigmoid')
    
    
    ])

In [48]:
nn1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 500)          5000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 493, 64)           256064    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 246, 64)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 246, 40)           2600      
_________________________________________________________________
dropout_3 (Dropout)          (None, 246, 40)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 246, 29)          

In [52]:
nn1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
nn1.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100)

Epoch 1/100
191/191 [==============================] - 4s 17ms/step - loss: 0.0962 - accuracy: 0.9768 - val_loss: 0.6154 - val_accuracy: 0.7485
Epoch 2/100
191/191 [==============================] - 3s 15ms/step - loss: 0.1017 - accuracy: 0.9762 - val_loss: 0.5608 - val_accuracy: 0.7531
Epoch 3/100
191/191 [==============================] - 3s 15ms/step - loss: 0.0937 - accuracy: 0.9775 - val_loss: 0.5775 - val_accuracy: 0.7511
Epoch 4/100
191/191 [==============================] - 3s 17ms/step - loss: 0.0945 - accuracy: 0.9773 - val_loss: 0.5669 - val_accuracy: 0.7505
Epoch 5/100
191/191 [==============================] - 3s 16ms/step - loss: 0.0984 - accuracy: 0.9767 - val_loss: 0.5468 - val_accuracy: 0.7531
Epoch 6/100
191/191 [==============================] - 3s 15ms/step - loss: 0.0964 - accuracy: 0.9767 - val_loss: 0.5439 - val_accuracy: 0.7584
Epoch 7/100
191/191 [==============================] - 3s 15ms/step - loss: 0.0943 - accuracy: 0.9778 - val_loss: 0.5733 - val_accuracy:

In [53]:
nn1.evaluate(x_train,y_train)

191/191 [==============================] - 1s 5ms/step - loss: 0.1671 - accuracy: 0.9793


[0.16705143451690674, 0.9793103337287903]

In [54]:
nn1.evaluate(x_test,y_test)

48/48 [==============================] - 0s 5ms/step - loss: 0.6746 - accuracy: 0.7479


[0.6745642423629761, 0.7478660345077515]

In [55]:
y_pred1=nn1.predict(x_test)
y_pred1=(y_pred1>0.5)
y_pred1[:5]

array([[False],
       [False],
       [ True],
       [False],
       [False]])

In [59]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_pred1=pd.DataFrame(y_pred1,columns=['pred1'])
y_pred1=le.fit_transform(y_pred1['pred1'])
y_pred1[:5]

array([0, 0, 1, 0, 0])

In [82]:
from sklearn.metrics import accuracy_score,classification_report,f1_score

In [62]:
accuracy_score(y_test,y_pred1)

0.747866053841103

In [85]:
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.76      0.82      0.79       875
           1       0.73      0.65      0.69       648

    accuracy                           0.75      1523
   macro avg       0.74      0.74      0.74      1523
weighted avg       0.75      0.75      0.75      1523



In [86]:
f1_score(y_test,y_pred1)

0.6878048780487804

# Applying another neural model by using Bidirectional LSTM

In [64]:
nn2=models.Sequential([
    
    layers.Embedding(voc_size,feature_vector,input_length=sent_length),
    layers.Dropout(0.5),
    layers.Bidirectional(layers.LSTM(199)),
    layers.Dropout(0.4),
    layers.Dense(399,activation='relu'),
    layers.Dropout(0.7),
    layers.Dense(43,activation='relu'),
    layers.Dropout(0.8),
    layers.Dense(1,activation='sigmoid')
    
])
    

In [65]:
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 500)          5000000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 500, 500)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 398)               1114400   
_________________________________________________________________
dropout_5 (Dropout)          (None, 398)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 399)               159201    
_________________________________________________________________
dropout_6 (Dropout)          (None, 399)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 43)               

In [67]:
nn2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
nn2.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10)

Epoch 1/10
191/191 [==============================] - 24s 109ms/step - loss: 0.0216 - accuracy: 0.9920 - val_loss: 5.2584 - val_accuracy: 0.7347
Epoch 2/10
191/191 [==============================] - 20s 105ms/step - loss: 0.0347 - accuracy: 0.9901 - val_loss: 4.4589 - val_accuracy: 0.7380
Epoch 3/10
191/191 [==============================] - 20s 105ms/step - loss: 0.0159 - accuracy: 0.9928 - val_loss: 5.9916 - val_accuracy: 0.7387
Epoch 4/10
191/191 [==============================] - 20s 105ms/step - loss: 0.0303 - accuracy: 0.9931 - val_loss: 3.6211 - val_accuracy: 0.7019
Epoch 5/10
191/191 [==============================] - 20s 105ms/step - loss: 0.0279 - accuracy: 0.9888 - val_loss: 3.8548 - val_accuracy: 0.7341
Epoch 6/10
191/191 [==============================] - 20s 106ms/step - loss: 0.0450 - accuracy: 0.9893 - val_loss: 3.9220 - val_accuracy: 0.7078
Epoch 7/10
191/191 [==============================] - 20s 105ms/step - loss: 0.0207 - accuracy: 0.9905 - val_loss: 4.1800 - val_ac

In [74]:
nn2.evaluate(x_train,y_train)

191/191 [==============================] - 7s 33ms/step - loss: 0.0127 - accuracy: 0.9943


[0.012666340917348862, 0.9942528605461121]

In [75]:
nn2.evaluate(x_test,y_test)

48/48 [==============================] - 2s 34ms/step - loss: 5.2737 - accuracy: 0.6967


[5.273706912994385, 0.6966513395309448]

# Applying ML models

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [76]:
lr=LogisticRegression(max_iter=150,random_state=28)
rfg=RandomForestClassifier(n_estimators=125,criterion='entropy')
gnb=GaussianNB()
xgb=XGBClassifier(n_estimators=199,learning_rate=0.05)
light=LGBMClassifier(learning_rate=0.001,n_estimators=101)
cat=CatBoostClassifier(iterations=600,learning_rate=0.01,loss_function='cross_entropy')

In [72]:
clf={'linear regression':lr,'random forest':rfg,'naive bayes':gnb,'xg boost':xgb,'light gbm':light,'cat boost':cat}

In [73]:
clf.keys()

dict_keys(['linear regression', 'random forest', 'naive bayes', 'xg boost', 'light gbm', 'cat boost'])

In [77]:
x

array([[   0,    0,    0, ..., 1007, 2529,   95],
       [   0,    0,    0, ..., 3492, 9123, 1894],
       [   0,    0,    0, ..., 8312, 7858, 7952],
       ...,
       [   0,    0,    0, ..., 4982,  327, 9457],
       [   0,    0,    0, ...,  730, 5403, 8862],
       [   0,    0,    0, ..., 4982, 2887, 4380]], dtype=int32)

In [78]:
y

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [80]:
x_test

array([[   0,    0,    0, ...,  450, 2352,  925],
       [   0,    0,    0, ..., 4785,  908, 5042],
       [   0,    0,    0, ..., 4982, 1725, 9152],
       ...,
       [   0,    0,    0, ..., 9881, 4982, 1411],
       [   0,    0,    0, ..., 9881, 4982, 3233],
       [   0,    0,    0, ..., 6108, 4051,  283]], dtype=int32)

In [96]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [97]:
x_train

array([[0.        , 0.        , 0.        , ..., 0.29508853, 0.24734841,
        0.25442721],
       [0.        , 0.        , 0.        , ..., 0.40112034, 1.        ,
        0.63491746],
       [0.        , 0.        , 0.        , ..., 0.5716715 , 0.91064639,
        0.15917959],
       ...,
       [0.        , 0.        , 0.        , ..., 0.89336801, 0.9023414 ,
        0.45262631],
       [0.        , 0.        , 0.        , ..., 0.4983495 , 0.13197919,
        0.90405203],
       [0.        , 0.        , 0.        , ..., 0.72451736, 0.17260356,
        0.01310655]])

In [98]:
x_test

array([[0.        , 0.        , 0.        , ..., 0.0450135 , 0.2353412 ,
        0.09244622],
       [0.        , 0.        , 0.        , ..., 0.47864359, 0.09085451,
        0.50435218],
       [0.        , 0.        , 0.        , ..., 0.4983495 , 0.17260356,
        0.91555778],
       ...,
       [0.        , 0.        , 0.        , ..., 0.98839652, 0.4984991 ,
        0.14107054],
       [0.        , 0.        , 0.        , ..., 0.98839652, 0.4984991 ,
        0.32336168],
       [0.        , 0.        , 0.        , ..., 0.61098329, 0.40534321,
        0.02821411]])

In [99]:
def classify(x,y):
    for key in clf.keys():
        clf[key].fit(x_train,y_train)
        y_pred2=clf[key].predict(x_test)
        score=f1_score(y_test,y_pred2)
        print(key,'--->',score)
        

In [100]:
classify(x,y)

linear regression ---> 0.49191489361702123
random forest ---> 0.5756336876533115
naive bayes ---> 0.6001855287569574
xg boost ---> 0.5702479338842975
light gbm ---> 0.0
0:	learn: 0.6921882	total: 9.41ms	remaining: 5.64s
1:	learn: 0.6911374	total: 16.7ms	remaining: 4.98s
2:	learn: 0.6901984	total: 23.4ms	remaining: 4.65s
3:	learn: 0.6893137	total: 30.2ms	remaining: 4.5s
4:	learn: 0.6883582	total: 36.5ms	remaining: 4.34s
5:	learn: 0.6875068	total: 43.4ms	remaining: 4.3s
6:	learn: 0.6866649	total: 50.2ms	remaining: 4.25s
7:	learn: 0.6858115	total: 57ms	remaining: 4.22s
8:	learn: 0.6849989	total: 63.3ms	remaining: 4.16s
9:	learn: 0.6841295	total: 70ms	remaining: 4.13s
10:	learn: 0.6832756	total: 76.8ms	remaining: 4.11s
11:	learn: 0.6824301	total: 83.6ms	remaining: 4.09s
12:	learn: 0.6816573	total: 90.5ms	remaining: 4.09s
13:	learn: 0.6809038	total: 97.2ms	remaining: 4.07s
14:	learn: 0.6801899	total: 104ms	remaining: 4.06s
15:	learn: 0.6794719	total: 111ms	remaining: 4.06s
16:	learn: 0.6787

In [101]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [102]:
test=test_df.copy()
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [107]:
test_df=test.copy()

In [104]:
test_df=test_df.drop(columns=['id','keyword','location'],axis=1)
test_df.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [105]:
len(test_df)

3263

In [111]:
test_df=test.copy()
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [113]:
corpus2=[]

for i in range(0,len(test_df)):
    print(i)
    review=re.sub('[^a-zA-Z]',' ',test_df['text'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus2.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [114]:
corpus2

['happen terribl car crash',
 'heard earthquak differ citi stay safe everyon',
 'forest fire spot pond gees flee across street cannot save',
 'apocalyps light spokan wildfir',
 'typhoon soudelor kill china taiwan',
 'shake earthquak',
 'probabl still show life arsen yesterday eh eh',
 'hey',
 'nice hat',
 'fuck',
 'like cold',
 'nooooooooo',
 'tell',
 '',
 'awesom',
 'birmingham wholesal market ablaz bbc news fire break birmingham wholesal market http co irwqcezweu',
 'sunkxssedharri wear short race ablaz',
 'previouslyondoyintv toke makinwa marriag crisi set nigerian twitter ablaz http co cmghxba xi',
 'check http co roi nsmejj http co tj zjin http co yduixefip http co lxtjc kl nsfw',
 'psa split person techi follow ablaz co burner follow ablaz',
 'bewar world ablaz sierra leon amp guap',
 'burn man ablaz turban diva http co hodwosamw via etsi',
 'diss song peopl take thing run smh eye open though set game ablaz cyhitheprync',
 'rape victim die set ablaz year old girl die burn injuri 

In [117]:
onehot_repr2=[text.one_hot(words,voc_size)for words in corpus2] 
onehot_repr2

[[3952, 9169, 27, 9088],
 [9948, 486, 2103, 6050, 8327, 8351, 5595],
 [5621, 5687, 3242, 3669, 267, 5161, 9698, 9193, 65, 8690],
 [3858, 1407, 7794, 5125],
 [5059, 1167, 7034, 8039, 9979],
 [1367, 486],
 [1102, 2684, 6775, 730, 7201, 1738, 1261, 1261],
 [6984],
 [7536, 934],
 [9411],
 [4785, 1680],
 [9811],
 [5109],
 [],
 [3508],
 [5324,
  2680,
  7877,
  3967,
  6133,
  283,
  5687,
  8129,
  5324,
  2680,
  7877,
  9881,
  4982,
  5020],
 [9305, 5780, 6603, 4174, 3967],
 [9594, 1999, 5443, 9472, 5775, 5008, 9582, 5166, 3967, 9881, 4982, 7495, 97],
 [1819,
  9881,
  4982,
  9849,
  7497,
  9881,
  4982,
  4419,
  8760,
  9881,
  4982,
  8318,
  9881,
  4982,
  2389,
  2647,
  9686],
 [5930, 5966, 996, 63, 4146, 3967, 4982, 6648, 4146, 3967],
 [9638, 2609, 3967, 6389, 686, 3745, 6321],
 [641, 385, 3967, 8373, 9367, 9881, 4982, 3646, 3590, 7190],
 [4933,
  5474,
  6506,
  1647,
  7625,
  9180,
  9933,
  6332,
  5880,
  8638,
  5008,
  9438,
  3967,
  2461],
 [4481,
  7336,
  4408,
  500

In [119]:
embedded_docs2=sequence.pad_sequences(onehot_repr2,padding='pre',maxlen=sent_length)
print(embedded_docs2)

[[   0    0    0 ... 9169   27 9088]
 [   0    0    0 ... 8327 8351 5595]
 [   0    0    0 ... 9193   65 8690]
 ...
 [   0    0    0 ... 9881 4982 7307]
 [   0    0    0 ... 4982 7910 4064]
 [   0    0    0 ... 4573 8809 4170]]


In [120]:
test_df=embedded_docs2.copy()
test_df

array([[   0,    0,    0, ..., 9169,   27, 9088],
       [   0,    0,    0, ..., 8327, 8351, 5595],
       [   0,    0,    0, ..., 9193,   65, 8690],
       ...,
       [   0,    0,    0, ..., 9881, 4982, 7307],
       [   0,    0,    0, ..., 4982, 7910, 4064],
       [   0,    0,    0, ..., 4573, 8809, 4170]], dtype=int32)

In [123]:
nn=models.Sequential([
    
    layers.Embedding(voc_size,feature_vector,input_length=sent_length),
    layers.Dropout(0.5),
    layers.Bidirectional(layers.LSTM(199)),
    layers.Dropout(0.4),
    layers.Dense(399,activation='relu'),
    layers.Dropout(0.7),
    layers.Dense(43,activation='relu'),
    layers.Dropout(0.8),
    layers.Dense(1,activation='sigmoid')
    
])
    

In [126]:
nn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
nn.fit(x,y,epochs=5)

Epoch 1/5
238/238 [==============================] - 27s 98ms/step - loss: 0.0153 - accuracy: 0.9926
Epoch 2/5
238/238 [==============================] - 23s 97ms/step - loss: 0.0156 - accuracy: 0.9925
Epoch 3/5
238/238 [==============================] - 23s 98ms/step - loss: 0.0218 - accuracy: 0.9921
Epoch 4/5
238/238 [==============================] - 23s 98ms/step - loss: 0.0182 - accuracy: 0.9926
Epoch 5/5
238/238 [==============================] - 23s 98ms/step - loss: 0.0258 - accuracy: 0.9917


In [128]:
nn.evaluate(x,y)

238/238 [==============================] - 9s 34ms/step - loss: 0.0074 - accuracy: 0.9959


[0.007359053939580917, 0.9959279894828796]

In [129]:
preds=nn.predict(test_df)
preds[:5]

array([[0.84715194],
       [1.        ],
       [0.9999385 ],
       [0.00143353],
       [1.        ]], dtype=float32)

In [130]:
preds=(preds>0.5)
preds[:5]

array([[ True],
       [ True],
       [ True],
       [False],
       [ True]])

In [131]:
preds=preds.reshape(-1,)
preds[:5]

array([ True,  True,  True, False,  True])

In [132]:
submission=pd.DataFrame({'id':test['id'],'target':preds})

In [133]:
submission.head()

,id,target
0,0,True
1,2,True
2,3,True
3,9,False
4,11,True


In [135]:
submission['target']=le.fit_transform(submission['target'])
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [137]:
submission['target'].value_counts(normalize=True)

0    0.539994
1    0.460006
Name: target, dtype: float64

In [138]:
submission.to_csv('nlp1.csv',index=None)